In [1]:
import dask.bag as db
from glob import glob
from dask.diagnostics import ProgressBar
import pickle
import re
import konlpy
# logging
import logging
logging.basicConfig(filename='exo.log',level=logging.INFO)
# Display progress
pbar = ProgressBar()
pbar.register()

# Load sentences to processing unit

In [2]:
fnames = glob('../data/sents/*.txt')
# fnames = fnames[:3]
b = db.from_filenames(fnames)

# Process 

In [9]:
wiki_terms = None
doosan_terms = None
with open('../data/dict/wiki_terms.pickle', 'rb') as fin:
    wiki_terms = pickle.load(fin)
with open('../data/dict/doosan_terms.pickle', 'rb') as fin:
    doosan_terms = pickle.load(fin)

In [6]:
def per_partition(lines):
    """
    위에서 사용한 `from_filenames`는 텍스트 파일을 문장 집합 형태로 로드한 후, 
    규모에 따라 수십개의 파티션으로 나눔
    예) [문장1, 문장2, 문장3, ...], [문장4, 문장5, 문장6, ...], ...
    
    파티션은 컴퓨터에서 활용 가능한 각 코어로 분배됨.
    
    이 함수에서는
    1) 파티션에 소속된 각 파일에 대해 원하는 처리를 수행
    2) 결과물을 반환
    """
    # Load the dictionary terms    
    wiki_terms = None
    doosan_terms = None
    with open('../data/dict/wiki_terms.pickle', 'rb') as fin:
        wiki_terms = pickle.load(fin)
    with open('../data/dict/doosan_terms.pickle', 'rb') as fin:
        doosan_terms = pickle.load(fin)
        
    # Initialize the necessary components 
    # before iterating over the sentences
    pattern1 = re.compile('(\"|\'|“|”|‘|’|《|》|\.|,|•|-)')
    pattern2 = re.compile('(·|-|\.)')
    tagger_twitter = konlpy.tag.Twitter()
    except_list = ["단계", "대해", "다소", "두", "세", "네", "오늘날", "하나하나", "정도", "처음","경", "자체", "후","하나", "앞", "뒤", "위","아래", "수","데", "내", "음", "번", "그후", "이하","이상", "여기", "권", "당시", "책", "년", "달", "월", "해", "일", "말", "초", "째", "사실상", "자신", "역시", "각종", "의", "앞", "지","속", "대", "전", "주요", "일", "제", "해도", "처", "이", "저", "수", "그", "때", "가지", "이후", "매우", "등", "못", "스스로", "오직", "이기", "볼", "초", "약", "중", "상", "개", "주", "예", "이전", "이번", "채", "안", "경우"]
        
    # Process each sentence
    # enumerate(): iterable의 인덱스를 함께 반환하는 helper 함수
    for i, line in enumerate(lines):
        try:
            sentence = line.split('\t')[3]
            original_sentence = sentence[:]
        except ValueError as e:
            logging.warning("error parsing %s" % sentence)
            continue
            
        # remove special characters (결과값이 예전과 약간 다를 수 있으니 검증 필요)
        ## 아예 제거
        sentence = pattern1.sub('', sentence)
        ## 공백으로 치환
        sentence = pattern2.sub(' ', sentence)

        # Perform tagging with twitter
        word_tag_list = tagger_twitter.pos(sentence)
        # (use list comprehension whenever possible)
        nouns = [word for word, pos in word_tag_list if pos in ('Alpha', 'Number', 'Noun')]
        
        # 결과값 반환
        return nouns
        
def aggregate(list_of_nouns):
    for noun_list in list_of_nouns:
        print(noun_list)

In [7]:
b2 = b.reduction(per_partition, aggregate)

In [8]:
b2.compute()

[                                        ] | 0% Completed |  3.2s['지미', '카터', '민주당', '출신', '미국', '39', '대통령']
['1792', '년', '영국', '시인', '퍼시', '비시', '셸리']
['2013', '년', '2014', '년', 'MBC', '우리', '일밤', '진짜', '사나이']
[########################################] | 100% Completed |  3.3s
